# Hardpoint breakaway tests recent history

In [ ]:
# Times Square parameters

days_to_plot = 30


In [ ]:
import asyncio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
def stiffness(force, disp, name):
    forces = force.values
    disps = disp.values
    if name == 'Pos':
        maxi = np.argmax(force.values > 1000.0)
        mini = np.argmin(force.values < -1000.0)
    elif name == 'Neg':
        maxi = np.argmin(force.values > 1000.0)
        mini = np.argmax(force.values < -1000.0)
    forceMax = forces[maxi] 
    forceMin = forces[mini]
    dispMax = disps[maxi] * 1.0E6
    dispMin = disps[mini] * 1.0E6
    stiffness = (forceMax - forceMin) / (dispMax - dispMin) 
    breakaway_plus = np.max(forces)
    breakaway_minus = np.min(forces)
    return breakaway_plus, breakaway_minus, stiffness


In [ ]:
end = Time.now()
start = end - TimeDelta(days_to_plot, format='jd')
client = EfdClient('usdf_efd')

hardpoints = await client.select_time_series(\
                    "lsst.sal.MTM1M3.logevent_hardpointTestStatus", \
                    ["*"], start, end)


In [ ]:
%matplotlib inline

states = [['Pos',3,4], ['Neg',4,5]]

fig, axs = plt.subplots(1,2,figsize=(10,5))
fig.suptitle("MTM1M3 hardpoint trending", fontsize=18)
axs[0].set_title("Breakaway forces", fontsize=14)
axs[0].set_ylim(-4000, 4000)
axs[0].set_ylabel("Breakaway force (N)")
axs[1].set_title("Stiffness", fontsize=14)
axs[1].set_ylim(0, 50)
axs[1].set_ylabel("Stiffness (N/micron)")
min_time = 1.0E12
max_time = 0.0

for [name, startState, endState] in states:
    for index in range(6):
        stiffs = []
        pos_breakaway = []
        neg_breakaway = []
        times = []
        getStarts = hardpoints[hardpoints[f'testState{index}']==startState]['private_kafkaStamp']
        getEnds = hardpoints[hardpoints[f'testState{index}']==endState]['private_kafkaStamp']
        for n in range(min(len(getStarts), len(getEnds))):
            start = Time(getStarts.iloc[n], format='unix_tai', scale='tai')
            end = Time(getEnds.iloc[n], format='unix_tai', scale='tai')
            try:
                hardpointData = await client.select_time_series(\
                    "lsst.sal.MTM1M3.hardpointActuatorData", \
                    [f'displacement{index}', f'measuredForce{index}'], start.utc, end.utc)

                force = hardpointData[f'measuredForce{index}']
                disp = hardpointData[f'displacement{index}']
                forceMin, forceMax, stiff = stiffness(force, disp, name)
                pos_breakaway.append(forceMax)
                neg_breakaway.append(forceMin)
                stiffs.append(stiff)
                time = start.utc.unix_tai
                times.append(time)
                if time < min_time:
                    min_time = time
                    min_time_list = start.isot.split('.')[0]
                if time > max_time:
                    max_time = time
                    max_time_list = start.isot.split('.')[0]
            except:
                continue

        if name == 'Pos':
            axs[0].plot(times, pos_breakaway, marker='x', label=f"{index}")
            axs[0].plot(times, neg_breakaway, marker='x', label=f"{index}")
        elif name == 'Neg':
            axs[0].plot(times, pos_breakaway, marker='x', label='')
            axs[0].plot(times, neg_breakaway, marker='x', label='')
        axs[1].plot(times, stiffs, marker='x', label=f"{name}_{index}")
        axs[0].set_xticks([min_time, max_time],[min_time_list, max_time_list], rotation=10)
        axs[1].set_xticks([min_time, max_time],[min_time_list, max_time_list], rotation=10)

axs[0].legend(loc='center left', ncol=3)
axs[1].legend(loc='upper left', ncol=3)
plt.show()
